In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import BertTokenizer#, TFBertForSequenceClassification
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.metrics import BinaryAccuracy
# import tensorflow as tf

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from sklearn.model_selection import train_test_split

In [2]:
#pip install tensorflow==2.11.0

In [3]:
miracl_dataset = load_dataset("miracl/miracl", "ru")
miracl_corpus = load_dataset("miracl/miracl-corpus", "ru")

In [4]:
print(miracl_dataset)
print(miracl_corpus)

DatasetDict({
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 1252
    })
    testB: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 718
    })
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 4683
    })
    testA: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 911
    })
})
DatasetDict({
    train: Dataset({
        features: ['docid', 'title', 'text'],
        num_rows: 9543918
    })
})


In [5]:
corpus_dict = {row["docid"]: row for row in miracl_corpus["train"]}

In [6]:
qa_pairs = []
labels = []

In [7]:
for split in ["train", "dev"]:  # Use both train and dev splits
    for example in miracl_dataset[split]:
        query = example["query"]
        # Add positive (relevant) pairs
        for pos_passage in example["positive_passages"]:
            pos_docid = pos_passage["docid"]  # Extract docid from the dictionary
            if pos_docid in corpus_dict:
                qa_pairs.append((query, corpus_dict[pos_docid]["text"]))
                labels.append(1)  # Relevant
        # Add negative (irrelevant) pairs
        for neg_passage in example["negative_passages"]:
            neg_docid = neg_passage["docid"]  # Extract docid from the dictionary
            if neg_docid in corpus_dict:
                qa_pairs.append((query, corpus_dict[neg_docid]["text"]))
                labels.append(0)  # Irrelevant

In [8]:
qa_df = pd.DataFrame(qa_pairs, columns=["query", "text"])
qa_df["label"] = labels

In [9]:
qa_df.head()

,query,text,label
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,1
1,Когда был спущен на воду первый миноносец «Спо...,Стерегу́щий — русский миноносец типа «Сокол». ...,0
2,Когда был спущен на воду первый миноносец «Спо...,Эскадренный миноносец заложен в 1900 году на Н...,0
3,Когда был спущен на воду первый миноносец «Спо...,10 октября 1937 года эсминец был спущен на вод...,0
4,Когда был спущен на воду первый миноносец «Спо...,В 1901 году миноносец «Бодрый» был зачислен в ...,0


In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [11]:
def preprocess(data):
    inputs = data["query"] + " [SEP] " + data["text"]
    model_inputs = tokenizer(inputs.tolist(), max_length=128, truncation=True, padding="max_length", return_tensors="np")
    labels = np.array(data["label"])
    model_inputs["labels"] = labels
    return model_inputs

In [12]:
preprocessed_data = preprocess(qa_df)

In [13]:
input_ids = preprocessed_data["input_ids"]
attention_mask = preprocessed_data["attention_mask"]
labels = preprocessed_data["labels"]

In [16]:
train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

In [6]:
#model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

NameError: name 'TFBertForSequenceClassification' is not defined

In [18]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from datasets import Dataset

# # Load the tokenizer and model for PyTorch
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# # Tokenize the dataset and prepare it for PyTorch
# def tokenize_data(examples):
#     return tokenizer(examples["query"], examples["text"], truncation=True, padding="max_length", max_length=128)

# # Create the Dataset from your question-answer pairs and labels
# qa_df["label"] = qa_df["label"].astype(int)  # Ensure labels are integers for PyTorch
# dataset = Dataset.from_pandas(qa_df)

# # Apply tokenization
# tokenized_dataset = dataset.map(tokenize_data, batched=True)

# # Split into train and test datasets
# train_dataset = tokenized_dataset.train_test_split(test_size=0.2)["train"]
# test_dataset = tokenized_dataset.train_test_split(test_size=0.2)["test"]

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",  # Evaluate at the end of each epoch
#     save_strategy="epoch",        # Save at the end of each epoch
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     load_best_model_at_end=True,  # Load the best model at the end of training
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# results = trainer.evaluate()
# print("Test Loss, Test Accuracy:", results)


In [ ]:
from sklearn.model_selection import StratifiedKFold
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from transformers import EarlyStoppingCallback


tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# Tokenize the dataset and prepare it for PyTorch
def tokenize_data(examples):
    return tokenizer(examples["query"], examples["text"], truncation=True, padding="max_length", max_length=128)

# Create the Dataset from your question-answer pairs and labels
qa_df["label"] = qa_df["label"].astype(int)  # Ensure labels are integers for PyTorch
dataset = Dataset.from_pandas(qa_df)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_data, batched=True)

# Предполагаем, что `tokenized_dataset` уже создан
skf = StratifiedKFold(n_splits=5)
results = []

# Подготовка аргументов для тренировки
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Установим больше эпох, чтобы дать возможность ранней остановки
    weight_decay=0.01,
    load_best_model_at_end=True,  # Загрузка лучшей модели в конце
)



# Метки для стратифицированного разбиения
labels = np.array(tokenized_dataset['label'])

# Кросс-валидация
for fold, (train_idx, test_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"Training fold {fold + 1}/5")
    
    # Разделение датасета на обучающую и тестовую выборки для текущего фолда
    train_dataset = tokenized_dataset.select(train_idx)
    test_dataset = tokenized_dataset.select(test_idx)

    # Создание модели для текущего фолда
    model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

    # Определение тренера с ранней остановкой
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Ранняя остановка после 3 эпох без улучшений
    )
    
    # Обучение модели
    trainer.train()

    # Оценка модели
    fold_results = trainer.evaluate()
    print(f"Fold {fold + 1} results:", fold_results)
    results.append(fold_results["eval_loss"])

# Средняя точность по всем фолдам
print("Average loss across all folds", np.mean(results))


In [25]:
model.save_pretrained("./relevance_10_model")
tokenizer.save_pretrained("./relevance_10_tokenizer")


('./relevance_10_tokenizer\\tokenizer_config.json',
 './relevance_10_tokenizer\\special_tokens_map.json',
 './relevance_10_tokenizer\\vocab.txt',
 './relevance_10_tokenizer\\added_tokens.json')

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("./relevance_model")
tokenizer = BertTokenizer.from_pretrained("./relevance_tokenizer")

In [14]:
from transformers import pipeline

# Set up a classification pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Combine question and answer with a separator
question = "Работает ли наша модель?"
answer = "Пожалуй."
combined_input = question + " [SEP] " + answer  # or simply question + " " + answer

# Run inference on the combined question-answer pair
result = classifier(combined_input)
print(result)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.5901882648468018}]
